# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 10:22AM,248256,16,8623533,Sartorius Bioprocess Solutions,Released
1,Oct 10 2022 10:18AM,248255,19,ACG-2102491524,ACG North America LLC,Released
2,Oct 10 2022 10:09AM,248239,10,PRF-21284,Bio-PRF,Released
3,Oct 10 2022 9:59AM,248254,10,MTR22-0259,"Senseonics, Incorporated",Released
4,Oct 10 2022 9:59AM,248254,10,MTR-22-0260,"Senseonics, Incorporated",Released
5,Oct 10 2022 9:21AM,248245,10,PRF-38785,Bio-PRF,Released
6,Oct 10 2022 9:21AM,248245,10,PRF-38788,Bio-PRF,Executing
7,Oct 10 2022 9:21AM,248245,10,PRF-38791,Bio-PRF,Released
8,Oct 10 2022 9:21AM,248245,10,PRF-38797,Bio-PRF,Released
9,Oct 10 2022 9:21AM,248245,10,PRF-38802,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,248245,Released,29
15,248246,Released,3
16,248254,Released,2
17,248255,Released,1
18,248256,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248245,NaN,10.0,29.0
248246,NaN,NaN,3.0
248254,NaN,NaN,2.0
248255,NaN,NaN,1.0
248256,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,4.0,18.0
248221,0.0,28.0,0.0
248228,0.0,0.0,11.0
248229,0.0,0.0,1.0
248235,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,4,18
248221,0.0,28,0
248228,0.0,0,11
248229,0.0,0,1
248235,0.0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,4,18
1,248221,0.0,28,0
2,248228,0.0,0,11
3,248229,0.0,0,1
4,248235,0.0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,4,18
1,248221,0.0,28,
2,248228,0.0,,11
3,248229,0.0,,1
4,248235,0.0,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated"
5,Oct 10 2022 9:21AM,248245,10,Bio-PRF
44,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions
47,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc."
50,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc."
79,Oct 10 2022 9:09AM,248243,12,LF of America Corp.
80,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions,0.0,,1
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC,0.0,,1
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF,0.0,,1
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",0.0,,2
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,0.0,10,29
5,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions,0.0,,3
6,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",0.0,,3
7,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",0.0,,29
8,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,0.0,,1
9,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation,0.0,21,41


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions,1,
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC,1,
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF,1,
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",2,
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,29,10
5,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions,3,
6,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,
7,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,
8,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,
9,Oct 10 2022 9:04AM,248236,10,ISDIN Corporation,41,21


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions,1,
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC,1,
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF,1,
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",2,
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,29,10


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions,1.0,NaN
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC,1.0,NaN
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF,1.0,NaN
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",2.0,NaN
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,29.0,10.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 10:22AM,248256,16,Sartorius Bioprocess Solutions,1.0,0.0
1,Oct 10 2022 10:18AM,248255,19,ACG North America LLC,1.0,0.0
2,Oct 10 2022 10:09AM,248239,10,Bio-PRF,1.0,0.0
3,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",2.0,0.0
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,29.0,10.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2482364,137.0,35.0
12,248243,1.0,0.0
16,496502,4.0,0.0
19,496476,1.0,28.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2482364,137.0,35.0
1,12,248243,1.0,0.0
2,16,496502,4.0,0.0
3,19,496476,1.0,28.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,137.0,35.0
1,12,1.0,0.0
2,16,4.0,0.0
3,19,1.0,28.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,137.0
1,12,Released,1.0
2,16,Released,4.0
3,19,Released,1.0
4,10,Executing,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,35.0,0.0,0.0,28.0
Released,137.0,1.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,35.0,0.0,0.0,28.0
1,Released,137.0,1.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,35.0,0.0,0.0,28.0
1,Released,137.0,1.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()